### Оценка модели

Соберем вместе все что мы имеем и реарганизуем это в чуть более сложные структуры

In [2]:
import re
negative_tweets = open('negative_tweets.txt', 'r').readlines()[:1000]
positive_tweets = open('positive_tweets.txt', 'r').readlines()[:1000]

def find_unique_words(tweets):
    clear_tweets = []
    words = []
    pattern = re.compile('[а-яА-Я]+')
    for tweet in tweets:
        res = re.findall(pattern, tweet)
        clear_tweets.append(' '.join(res))
        words.extend(res)
    words = set(words)
    return clear_tweets, words

clear_pos_tweets, pos_words = find_unique_words(positive_tweets)
clear_neg_tweets, neg_words = find_unique_words(negative_tweets)
inter = pos_words.intersection(neg_words)
pos_words = pos_words - inter
neg_words = neg_words - inter

Так как мы имеем два независмых набора слов, мы можем попытаться оценить новые твиты с их помощью. Это будет наша тестовая выборка 

In [3]:
test_nt = open('negative_tweets.txt', 'r').readlines()[1000:1025]
test_pt = open('positive_tweets.txt', 'r').readlines()[1000:1025]
test_cpt, _ = find_unique_words(test_pt)
test_cnt, _ = find_unique_words(test_nt)

Теперь у нас есть небольшая тествая выборка из 50 примеров, где и положительных и отрицательных твитов поровну. Постараемся оценить эти твиты по среднему значению. Разделим твиты на слова и если слово является маркером положительным добавим к оценке 1, если отрицательным отнимем единицу и чтобы привести все к интервалу от 0 до 1, будем делить на количество слов в предложении. Что и происходит в нижеследующем коде.

In [6]:
def measure_score(test):
    score_list=[]
    for tweet in test:
        score = 0
        for word in tweet.split(' '):
            if word in pos_words:
                score+=1
            elif word in neg_words:
                score-=1
        score/=len(tweet.split(' '))
        score_list.append(score)
    return score_list

scores_pos = measure_score(test_cpt)
scores_neg = measure_score(test_cnt)
print(scores_neg, scores_pos)

[-0.07692307692307693, 0.0, 0.09090909090909091, 0.0, 0.0, -0.06666666666666667, 0.0, 0.25, -0.16666666666666666, -0.18181818181818182, 0.0, -0.16666666666666666, -0.3333333333333333, -0.17647058823529413, 0.0, 0.0, -0.14285714285714285, 0.0, 0.0, 0.0, -0.25, -0.1, 0.0, 0.0, 0.0] [0.0, 0.0, 0.0, 0.0, 0.375, 0.0, 1.0, 0.14285714285714285, -0.08333333333333333, 0.0, 0.0, 0.3333333333333333, 0.0, -0.09090909090909091, 0.0, 0.0, 0.0, 0.0, -0.07142857142857142, -0.1111111111111111, 1.0, -0.5, 1.0, 0.5, 0.0]


Как мы можем отметить, большинство твитов не имеем никакого коэфицента. Давайте попробуем объяснить почему так произошло и какими простыми методами мы можем поправить эту ситауцию?

### True Positive, True Negative, False Positive, False Negative, Accuracy and Precision

Наши устные оценки хороши, однако неплохо бы привести к обще известным метрикам и в целом понять как они устроены. В конце концов нам нужно меть возможность не только сравнивать свои модели, но и модели уже существующие.

<img src='https://cdn-images-1.medium.com/max/1600/1*CPnO_bcdbE8FXTejQiV2dg.png' >

Теперь вернемся к нашей задаче, что мы будем считать:
<br> *True Positive*
<br> *True Negative*
<br> *False Positive*
<br> *False Negative*?

Сейчас у нас есть два списка с предсказанными значениями, и а так как мы знаем какие твиты позитивные, а какие нет, что может создать еще два списка с *истинными* значениями.

In [8]:
true_pos_scores = [1 for i in range(25)] #заодно познакомимся с вложенным списками и простой их генерацией в питоне
true_neg_scores = [-1 for i in range(25)]
print(true_neg_scores, true_pos_scores)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


В идеале наши модель должна генерировать именно такие знаения. Чувствуете разницу? Но будем к ней так критичны и решим так: если предсказанное значение больше 0, значит модель считает твит позитивным, если меньше 0 - негативным. Пройдемся по предсказанным значениям и посчитаем сколько же мы имеем TP, TN, FP and FN.

In [10]:
TP= len([1 for i in scores_pos if i>0]) 
TN= len([1 for i in scores_neg if i<0])
FP= len([1 for i in scores_neg if i>0])
FN= len([1 for i in scores_pos if i<0])
print((TP+TN+FP+FN)/50)

0.48


Как вы считаете, что показывает нам это значение?

Несмотря на то, что модель не смогла предсказать значения для всех твитов, мы можем посчитать точность и полноту.
<img src='https://www.shmula.com/wp-content/uploads/2010/05/accuracy-precision-msa-shmula.jpg' >

Немного формул:
$$Accuracy = (TP+TN)/(TP+TN+FP+FN)$$
$$Precision = TP/(TP+FP)$$

In [12]:
accuracy = (TP+TN)/(TP+TN+FP+FN)
prec = (TP)/(TP+FP)
print('Accuracy: ', accuracy, 'Precision: ', prec)

Accuracy:  0.7083333333333334 Precision:  0.7777777777777778


Неплохие результаты, если не считать, что в половину твитов мы не оценили никаким образом.